# `jupyterlite` blog integration

    hatch run lite:build
    

i typically write content in my local environments.  there is a different style required to make the same documents reusable in `jupyterlite`. 

this document is a doit task that builds a lot site and ammends content for lite.

In [27]:
    import tonyfast, pathlib, textwrap, re, json
    __import__("requests_cache").install_cache()
    WHERE = pathlib.Path(tonyfast.__file__).parent.parent
    LITE = WHERE / "site/run"
    FILES = LITE / "files"

In [28]:
    PIDGY = re.compile("^\s*%(re)?load_ext\s*pidgy")
    from midgy import Python
    tangle = Python()

In [30]:
    def iter_code_cells(nb):
        for i, cell in enumerate(nb["cells"]):
            if cell["cell_type"] == "code":
                yield i, cell

In [31]:
    def has_pidgy(nb):
        yes = False
        for _, cell in iter_code_cells(nb):
            yes = yes or PIDGY.match("".join(cell["source"])) and True
        return yes

In [38]:
    def get_imports(cell, pidgy=False):
        import depfinder
        source = "".join(cell["source"])
        if pidgy:
            source = tangle.render(source)
        source = textwrap.dedent(source)
        try:
            found = depfinder.inspection.get_imported_libs(source)
            return found.required_modules.union(found.sketchy_modules)
        except BaseException as e:
            return

In [86]:
    def get_deps(deps):
        if "requests" in deps:
            deps.add("pyodide-http")
        if "pandas" in deps:
            deps.add("jinja2")
        return {
            x for x in deps 
            if not x.startswith("_") or x not in {"tonyfast"}
        }

In [79]:
    def set_file_imports(file):
        data = json.loads(file.read_text())
        deps, first = set(), None
        pidgy = has_pidgy(data)
        for no, cell in iter_code_cells(data):
            if first is None:
                first = no
            deps.update(get_imports(cell, pidgy) or [])
            
        deps = get_deps(deps)
        if deps and (first is not None):
            cell = data["cells"][first]
            was_str = isinstance(cell["source"], str)
            if was_str:
                cell["source"] = cell["source"].splitlines(1)
            
            for i, line in enumerate(list(cell["source"])):
                if (left := line.lstrip()):
                    if left.startswith(("%pip install",)):
                        break
                    indent = len(line) - len(left)                    
                    if "pyodide-http" in deps:
                        data["cells"][first]["source"].insert(i, " "*indent + "__import__('pyodide_http').patch_all()\n")
                    data["cells"][first]["source"].insert(i, " "*indent + "%pip install " + " ".join(deps) +"\n")
                    print(F"writing {len(set(deps))} pip requirements to {file}")
                    file.write_text(json.dumps(data, indent=2))
                    break
        else:
            print(F'no deps for {file}')

In [80]:
    def set_files_imports(FILES=FILES):
        for file in FILES.rglob("*.ipynb"):
            set_file_imports(file)
            

In [81]:
    def task_lite():
        """build the jupyter lite site and append requirements"""
        return dict(
            actions=[
                "jupyter lite build --contents tonyfast --output-dir site/run",
                (set_files_imports, (pathlib.Path("site/run/files"),))
            ],
            clean=["rm -rf site/run"]
        )

In [82]:
    if "__file__" not in locals():
        !importnb -t 2022-12-21-lite-build.ipynb list

lite   build the jupyter lite site and append requirements
